In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from optimus import Optimus

# Create optimus
op = Optimus(dl=True)

df = op.read.csv("random.csv",header=True, sep=";")

Using TensorFlow backend.


In [5]:
df.table()

NameError: name 'df' is not defined

## Key Collision

In [53]:
from optimus.ml import keycollision as keyCol


op.spark.conf.set("spark.sql.shuffle.partitions", "4")
op.spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 1 * 1024 * 1024 * 1024)

keyCol.fingerprint(df, 'STATE').table(columns=["STATE","STATE_FINGERPRINT"])

STATE 1 (string),STATE_FINGERPRINT 2 (string)
Distrito⸱Federal,distritofederal
Distrito⸱Federal,distritofederal
Distrito⸱Federal,distritofederal
Distrito⸱Federal,distritofederal
Distrito⸱Federal,distritofederal
Distrito⸱Federal,distritofederal
Distrito⸱Federal,distritofederal
Distrito⸱Federal,distritofederal
Distrito⸱Federal,distritofederal
Distrito⸱Federal,distritofederal


In [54]:
keyCol.fingerprint_cluster(df, 'STATE').table()

cluster_size 1 (int),cluster 2 (array<string>),count 3 (bigint),recommended 4 (string)
2,"['México⸱D.F.',⸱'Mexico⸱D.F.']",2495,México⸱D.F.
1,['Distriro⸱Federal'],259,Distriro⸱Federal
3,"['Distrito⸱Federal',⸱'DISTRITO⸱FEDERAL',⸱'distrito⸱federal']",11930,Distrito⸱Federal
1,['D.F.'],66,D.F.
1,['Estado⸱de⸱México'],810,Estado⸱de⸱México


In [55]:
keyCol.n_gram_fingerprint(df, 'STATE', 2).table(columns=["STATE","STATE_NGRAM", "STATE_NGRAM_FINGERPRINT"])

STATE 1 (string),STATE_NGRAM 2 (array<string>),STATE_NGRAM_FINGERPRINT 3 (string)
Distrito⸱Federal,"['d',⸱'i',⸱'s',⸱'t',⸱'r',⸱'i',⸱'t',⸱'o',⸱'f',⸱'e',⸱'d',⸱'e',⸱'r',⸱'a',⸱'l',⸱'']",aldediederfeisitlofraristtotr
Distrito⸱Federal,"['d',⸱'i',⸱'s',⸱'t',⸱'r',⸱'i',⸱'t',⸱'o',⸱'f',⸱'e',⸱'d',⸱'e',⸱'r',⸱'a',⸱'l',⸱'']",aldediederfeisitlofraristtotr
Distrito⸱Federal,"['d',⸱'i',⸱'s',⸱'t',⸱'r',⸱'i',⸱'t',⸱'o',⸱'f',⸱'e',⸱'d',⸱'e',⸱'r',⸱'a',⸱'l',⸱'']",aldediederfeisitlofraristtotr
Distrito⸱Federal,"['d',⸱'i',⸱'s',⸱'t',⸱'r',⸱'i',⸱'t',⸱'o',⸱'f',⸱'e',⸱'d',⸱'e',⸱'r',⸱'a',⸱'l',⸱'']",aldediederfeisitlofraristtotr
Distrito⸱Federal,"['d',⸱'i',⸱'s',⸱'t',⸱'r',⸱'i',⸱'t',⸱'o',⸱'f',⸱'e',⸱'d',⸱'e',⸱'r',⸱'a',⸱'l',⸱'']",aldediederfeisitlofraristtotr
Distrito⸱Federal,"['d',⸱'i',⸱'s',⸱'t',⸱'r',⸱'i',⸱'t',⸱'o',⸱'f',⸱'e',⸱'d',⸱'e',⸱'r',⸱'a',⸱'l',⸱'']",aldediederfeisitlofraristtotr
Distrito⸱Federal,"['d',⸱'i',⸱'s',⸱'t',⸱'r',⸱'i',⸱'t',⸱'o',⸱'f',⸱'e',⸱'d',⸱'e',⸱'r',⸱'a',⸱'l',⸱'']",aldediederfeisitlofraristtotr
Distrito⸱Federal,"['d',⸱'i',⸱'s',⸱'t',⸱'r',⸱'i',⸱'t',⸱'o',⸱'f',⸱'e',⸱'d',⸱'e',⸱'r',⸱'a',⸱'l',⸱'']",aldediederfeisitlofraristtotr
Distrito⸱Federal,"['d',⸱'i',⸱'s',⸱'t',⸱'r',⸱'i',⸱'t',⸱'o',⸱'f',⸱'e',⸱'d',⸱'e',⸱'r',⸱'a',⸱'l',⸱'']",aldediederfeisitlofraristtotr
Distrito⸱Federal,"['d',⸱'i',⸱'s',⸱'t',⸱'r',⸱'i',⸱'t',⸱'o',⸱'f',⸱'e',⸱'d',⸱'e',⸱'r',⸱'a',⸱'l',⸱'']",aldediederfeisitlofraristtotr


In [57]:
keyCol.n_gram_fingerprint_cluster(df, "STATE" , 2).table()

cluster_size 1 (int),cluster 2 (array<string>),count 3 (bigint),recommended 4 (string)
2,"['México⸱D.F.',⸱'Mexico⸱D.F.']",2495,México⸱D.F.
1,['D.F.'],66,D.F.
3,"['Distrito⸱Federal',⸱'DISTRITO⸱FEDERAL',⸱'distrito⸱federal']",11930,Distrito⸱Federal
1,['Estado⸱de⸱México'],810,Estado⸱de⸱México
1,['Distriro⸱Federal'],259,Distriro⸱Federal


In [58]:
keyCol.to_json(df, "STATE" , 2)

[{'cluster_size': 2,
  'cluster': ['México D.F.', 'Mexico D.F.'],
  'count': 2495,
  'recommended': 'México D.F.'},
 {'cluster_size': 1, 'cluster': ['D.F.'], 'count': 66, 'recommended': 'D.F.'},
 {'cluster_size': 3,
  'cluster': ['Distrito Federal', 'DISTRITO FEDERAL', 'distrito federal'],
  'count': 11930,
  'recommended': 'Distrito Federal'},
 {'cluster_size': 1,
  'cluster': ['Estado de México'],
  'count': 810,
  'recommended': 'Estado de México'},
 {'cluster_size': 1,
  'cluster': ['Distriro Federal'],
  'count': 259,
  'recommended': 'Distriro Federal'}]

## Distance Cluster

In [59]:
from optimus.ml import distancecluster as dc
dc.levenshtein_matrix(df,"STATE").table()

STATE_LEVENSHTEIN_1 1 (string),STATE_LEVENSHTEIN_2 2 (string),STATE_LEVENSHTEIN_DISTANCE 3 (int)
mexicodf,mexicodf,0
mexicodf,distrirofederal,12
mexicodf,distritofederal,12
mexicodf,df,6
mexicodf,estadodemexico,10
distrirofederal,mexicodf,12
distrirofederal,distrirofederal,0
distrirofederal,distritofederal,1
distrirofederal,df,13
distrirofederal,estadodemexico,11


In [60]:
dc.levenshtein_filter(df,"STATE").table()

STATE_FROM 1 (string),STATE_LEVENSHTEIN_DISTANCE 2 (int),STATE_TO 3 (string)
df,6,mexicodf
mexicodf,6,df
distrirofederal,1,distritofederal
distritofederal,1,distrirofederal
estadodemexico,10,mexicodf


In [61]:
df_l = dc.levenshtein_cluster(df,"STATE")

In [62]:
dc.to_json(df, "STATE")

[{'cluster': ['D.F.'], 'cluster_size': 1, 'recommended': 'D.F.', 'count': 66},
 {'cluster': ['México D.F.', 'Mexico D.F.'],
  'cluster_size': 2,
  'recommended': 'México D.F.',
  'count': 2495},
 {'cluster': ['Distriro Federal'],
  'cluster_size': 1,
  'recommended': 'Distriro Federal',
  'count': 259},
 {'cluster': ['Distrito Federal', 'DISTRITO FEDERAL', 'distrito federal'],
  'cluster_size': 3,
  'recommended': 'Distrito Federal',
  'count': 11930},
 {'cluster': ['Estado de México'],
  'cluster_size': 1,
  'recommended': 'Estado de México',
  'count': 810}]

## Feature Engineering

In [63]:
data = [('Japan', 'Tokyo', 37800000),('USA', 'New York', 19795791),('France', 'Paris', 12341418),
              ('Spain','Madrid',6489162)]
df = op.spark.createDataFrame(data, ["country", "city", "population"])

In [64]:
df.table()

country 1 (string),city 2 (string),population 3 (bigint)
Japan,Tokyo,37800000
USA,New⸱York,19795791
France,Paris,12341418
Spain,Madrid,6489162


In [65]:
from optimus.ml import feature as fe

### String to Index

In [66]:
df_sti = fe.string_to_index(df, input_cols=["city", "country"])

In [67]:
df_sti.table()

country 1 (string),city 2 (string),population 3 (bigint),country_index 4 (double),city_index 5 (double)
Japan,Tokyo,37800000,3.0,2.0
USA,New⸱York,19795791,2.0,3.0
France,Paris,12341418,1.0,0.0
Spain,Madrid,6489162,0.0,1.0


### Index to string

In [68]:
# Going back to strings from index
df_its = fe.index_to_string(df_sti, input_cols=["country_index"])

# Show DF with column "county_index" back to string
df_its.table()

country 1 (string),city 2 (string),population 3 (bigint),country_index 4 (double),city_index 5 (double),country_index_string 6 (string)
Japan,Tokyo,37800000,3.0,2.0,Japan
USA,New⸱York,19795791,2.0,3.0,USA
France,Paris,12341418,1.0,0.0,France
Spain,Madrid,6489162,0.0,1.0,Spain


### One Hot Enconder

In [69]:
# Creating DataFrame
data = [
(0, "a"),
(1, "b"),
(2, "c"),
(3, "a"),
(4, "a"),
(5, "c")
]
df = op.spark.createDataFrame(data,["id", "category"])

# One Hot Encoding
df_ohe = fe.one_hot_encoder(df, input_cols=["id"])

# Show encoded dataframe
df_ohe.table()

id 1 (bigint),category 2 (string),id_encoded 3 (vector)
0,a,"(5,[0],[1.0])"
1,b,"(5,[1],[1.0])"
2,c,"(5,[2],[1.0])"
3,a,"(5,[3],[1.0])"
4,a,"(5,[4],[1.0])"
5,c,"(5,[],[])"


### Vector assembler

In [25]:
# Import Vectors
from pyspark.ml.linalg import Vectors

# Creating DataFrame
data = [(0, 18, 1.0, Vectors.dense([0.0, 10.0, 0.5]), 1.0)]

df = op.spark.createDataFrame(data,["id", "hour", "mobile", "user_features", "clicked"])

# Assemble features
df_va = fe.vector_assembler(df, input_cols=["hour", "mobile", "user_features"])


# Show assembled df
print("Assembled columns 'hour', 'mobile', 'user_features' to vector column 'features'")
df_va.select("features", "clicked").table()

Assembled columns 'hour', 'mobile', 'userFeatures' to vector column 'features'


features (vector),clicked (double)
"[18.0,1.0,0.0,10.0,0.5]",1.0


### Normalizer

In [8]:
# Import Vectors
from pyspark.ml.linalg import Vectors

data = [
(0, Vectors.dense([1.0, 0.5, -1.0]),),
(1, Vectors.dense([2.0, 1.0, 1.0]),),
(2, Vectors.dense([4.0, 10.0, 2.0]),)
]

df = op.spark.createDataFrame(data,["id", "features"])

df_norm = fe.normalizer(df, input_cols=["features"], p=2.0)

df_norm.table()

id (bigint),features (vector),features_normalized (vector)
0,"[1.0,0.5,-1.0]","[0.666666666667,0.333333333333,-0.666666666667]"
1,"[2.0,1.0,1.0]","[0.816496580928,0.408248290464,0.408248290464]"
2,"[4.0,10.0,2.0]","[0.36514837167,0.912870929175,0.182574185835]"
